In [13]:
!pip install --upgrade pip -q
!pip install opentrain datasets -q

In [14]:
from datasets import load_dataset

In [38]:
ds = load_dataset("imdb", split="train").shuffle(seed=42)

Found cached dataset imdb (/Users/alvarobartt/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached shuffled indices for dataset at /Users/alvarobartt/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-9c48ce5d173413c7.arrow


In [39]:
label2idx = {label: idx for idx, label in enumerate(ds.features["label"].names)}
idx2label = {idx: label for label, idx in label2idx.items()}

In [40]:
ds = ds.select(range(1000))

In [41]:
PROMPT_TEMPLATE = "Categorize the following text from an IMDB review in the following categories based on its sentiment: 'pos', or 'neg'.\n\nReview: {text}\n\nLabel: "

In [42]:
def text_and_label_to_prompt_and_completion(example) -> dict:
    return {
        "prompt": PROMPT_TEMPLATE.format(text=example["text"]),
        "completion": idx2label[example["label"]],
    }

In [43]:
ds = ds.map(text_and_label_to_prompt_and_completion, remove_columns=ds.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [44]:
ds.to_json("imdb.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1495356

In [69]:
import openai

openai.api_key = "<ADD_YOUR_OPENAI_API_KEY_HERE>"

In [54]:
from opentrain.train import OpenAITrainer

In [55]:
trainer = OpenAITrainer(model="curie")

In [56]:
fine_tune_id = trainer.train("imdb.jsonl", epochs=5, batch_size=64)

/var/folders/tq/d6srhsd134l2b6x4sj01hpqr0000gn/T/ipykernel_14681/526391229.py:1: UserWarning: Since the OpenAI API may take from minutes to hours depending on the size of the training data, then from now on, you'll be able to check its progress via the following command: `openai api fine_tunes.follow -i ft-nbCAuynTlICBi7HfJ82XFB9F`. Once the training is completed, then you'll be able to use `OpenAIPredict` with the either the fine tune id returned, or from the model name generated by OpenAI linked to your account.
  fine_tune_id = trainer.train("imdb.jsonl", epochs=5, batch_size=64)


In [57]:
fine_tune_id

'ft-nbCAuynTlICBi7HfJ82XFB9F'

In [60]:
for event in trainer.track_training():
    print(event)

{
  "created_at": 1681907691,
  "level": "info",
  "message": "Created fine-tune: ft-nbCAuynTlICBi7HfJ82XFB9F",
  "object": "fine-tune-event"
}
{
  "created_at": 1681907801,
  "level": "info",
  "message": "Fine-tune costs $4.99",
  "object": "fine-tune-event"
}
{
  "created_at": 1681907801,
  "level": "info",
  "message": "Fine-tune enqueued. Queue number: 0",
  "object": "fine-tune-event"
}
{
  "created_at": 1681907802,
  "level": "info",
  "message": "Fine-tune started",
  "object": "fine-tune-event"
}
{
  "created_at": 1681907902,
  "level": "info",
  "message": "Completed epoch 1/5",
  "object": "fine-tune-event"
}
{
  "created_at": 1681907935,
  "level": "info",
  "message": "Completed epoch 2/5",
  "object": "fine-tune-event"
}
{
  "created_at": 1681907968,
  "level": "info",
  "message": "Completed epoch 3/5",
  "object": "fine-tune-event"
}
{
  "created_at": 1681908000,
  "level": "info",
  "message": "Completed epoch 4/5",
  "object": "fine-tune-event"
}
{
  "created_at": 168

In [61]:
from opentrain.predict import OpenAIPredict

In [62]:
predict = OpenAIPredict(fine_tune_id=fine_tune_id)

In [63]:
ds = load_dataset("imdb", split="test").shuffle(seed=42)

Found cached dataset imdb (/Users/alvarobartt/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [67]:
completion = predict(PROMPT_TEMPLATE.format(text=ds[0]["text"]))

In [68]:
idx2label[ds[0]["label"]] == completion

True